# GIF das 73 Pentadas do Ano

In [2]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import PillowWriter
import numpy as np
from tqdm import tqdm

import os
from dotenv import load_dotenv

from loczcit_iqr.core.data_loader_era5 import ERA5DataLoader 
from loczcit_iqr.core.processor import DataProcessor
from loczcit_iqr.core.iqr_detector import IQRDetector
from loczcit_iqr.core.spline_interpolator import (
    SplineInterpolator, 
    SplineParameters, 
    InterpolationMethod
)
from loczcit_iqr.plotting.visualizer import ZCITVisualizer
from loczcit_iqr.utils import pentada_to_dates

# ============================================================================
# CONFIGURAÇÃO INICIAL (igual ao seu código)
# ============================================================================
CDS_URL = "https://cds.climate.copernicus.eu/api"

# Carregar credenciais
dotenv_path = os.path.join('notebooks', '.env')
load_dotenv(dotenv_path=dotenv_path)
CDS_KEY = os.getenv("CDS_KEY")

# ============================================================================
# 1. CARREGAR DADOS DO ANO COMPLETO (executar apenas uma vez)
# ============================================================================
ano = 2024

# Inicializar componentes
loader = ERA5DataLoader(cds_url=CDS_URL, cds_key=CDS_KEY)
processor = DataProcessor()
detector = IQRDetector(constant=0.75)
interpolator = SplineInterpolator()
viz = ZCITVisualizer(template='publication')

print(f"Carregando dados ERA5 para o ano {ano}...")
olr_data = loader.load_data(
    start_date=f"{ano}-01-01", 
    end_date=f"{ano}-12-31"
)

print("Criando pentadas...")
pentads_year = processor.create_pentads(
    olr_data=olr_data, 
    year=ano,
    remove_leap_days=True,
    method='mean',
    min_days_required=3
)

# ============================================================================
# 2. PROCESSAR TODAS AS 73 PENTADAS
# ============================================================================
print("\n🎬 Iniciando processamento das 73 pentadas...")

# Lista para armazenar os dados de cada pentada
frames_data = []

# Configuração dos parâmetros de interpolação (usar para todas as pentadas)
params_bspline = SplineParameters(
    method=InterpolationMethod.BSPLINE,
    smooth_factor='high',
    degree=3,
    max_curvature_threshold=30,
    num_points_output=100,
    extrapolate_flag=True,
    reference_latitude=0
)

# Processar cada pentada
for pentada in tqdm(range(1, 74), desc="Processando pentadas"):
    try:
        # Extrair dados da pentada
        olr_pentada = pentads_year['olr'].sel(pentada=pentada)
        
        # Encontrar coordenadas mínimas
        min_coords = processor.find_minimum_coordinates(
            olr_pentada,
            threshold=None,
            method='column_minimum',
            search_radius=1
        )
        
        # Detectar outliers
        coords_valid, coords_outliers, resumo = detector.detect_outliers(min_coords)
        
        # Encontrar sistemas convectivos
        sistemas_convectivos = processor.find_minimum_coordinates(
            olr_pentada,
            threshold=230,
            method='local_minimum',
            search_radius=1
        )
        
        # Interpolar linha da ZCIT
        zcit_line, _ = interpolator.interpolate(coords_valid, parameters=params_bspline)
        
        # Criar título com datas
        start_date, end_date = pentada_to_dates(pentada, ano)
        titulo = (
            f"Análise ZCIT - Pentada {pentada} "
            f"({start_date.strftime('%d/%m')} - {end_date.strftime('%d/%m/%Y')})"
        )
        
        # Armazenar dados processados
        frames_data.append({
            'pentada': pentada,
            'olr_data': olr_pentada,
            'titulo': titulo,
            'coords_valid': coords_valid,
            'coords_outliers': coords_outliers,
            'sistemas_convectivos': sistemas_convectivos,
            'zcit_line': zcit_line
        })
        
    except Exception as e:
        print(f"\n⚠️ Erro na pentada {pentada}: {e}")
        continue

print(f"\n✅ {len(frames_data)} pentadas processadas com sucesso!")

# ============================================================================
# 3. SALVAR FRAMES INDIVIDUAIS
# ============================================================================
print("\n📸 Salvando frames individuais...")

# Criar diretório para frames temporários
frames_dir = "frames_temp"
os.makedirs(frames_dir, exist_ok=True)

# Salvar cada frame como imagem
for i, frame_data in enumerate(tqdm(frames_data, desc="Salvando frames")):
    # Criar visualização SEM o parâmetro 'show'
    fig_frame, ax_frame = viz.plot_complete_analysis(
        olr_data=frame_data['olr_data'],
        title=frame_data['titulo'],
        coords_valid=frame_data['coords_valid'],
        coords_outliers=frame_data['coords_outliers'],
        sistemas_convectivos=frame_data['sistemas_convectivos'],
        zcit_line=frame_data['zcit_line'],
        study_area_visible=True,
        credits="ERA5",
        save_path=None  # Não salvar automaticamente
    )
    
    # Adicionar indicador de pentada
    ax_frame.text(0.02, 0.98, f"Pentada {frame_data['pentada']}/73", 
                  transform=ax_frame.transAxes, 
                  fontsize=12, 
                  verticalalignment='top',
                  bbox=dict(boxstyle="round,pad=0.5", 
                           facecolor="yellow", 
                           alpha=0.8))
    
    # Salvar frame
    frame_path = os.path.join(frames_dir, f"frame_{i:03d}.png")
    fig_frame.savefig(frame_path, dpi=100, bbox_inches='tight')
    plt.close(fig_frame)  # Importante: fechar a figura para economizar memória

# ============================================================================
# 4. CRIAR GIF ANIMADO
# ============================================================================
print("\n🎬 Criando GIF animado...")

# Tentar primeiro com imageio (mais eficiente)
try:
    import imageio
    
    # Ler todos os frames
    images = []
    for i in range(len(frames_data)):
        frame_path = os.path.join(frames_dir, f"frame_{i:03d}.png")
        images.append(imageio.imread(frame_path))
    
    # Criar GIF
    output_path = f"zcit_evolution_{ano}_all_pentads.gif"
    imageio.mimsave(output_path, images, 
                    duration=500,  # 500ms por frame (0.5 segundo)
                    loop=0)  # Loop infinito
    
    print(f"\n✅ GIF salvo como: {output_path}")
    
except ImportError:
    print("\n⚠️ imageio não instalado. Usando método alternativo com PIL...")
    
    # Método alternativo com PIL/Pillow
    try:
        from PIL import Image
        
        # Carregar todos os frames
        frames = []
        for i in range(len(frames_data)):
            frame_path = os.path.join(frames_dir, f"frame_{i:03d}.png")
            img = Image.open(frame_path)
            frames.append(img)
        
        # Salvar como GIF
        output_path = f"zcit_evolution_{ano}_all_pentads.gif"
        frames[0].save(
            output_path,
            save_all=True,
            append_images=frames[1:],
            duration=500,  # 500ms por frame
            loop=0  # Loop infinito
        )
        
        print(f"\n✅ GIF salvo como: {output_path}")
        
    except ImportError:
        print("\n❌ Nem imageio nem PIL estão instalados!")
        print("Instale um deles com:")
        print("  pip install imageio")
        print("  ou")
        print("  pip install Pillow")
        print("\nOs frames individuais foram salvos em:", frames_dir)

# ============================================================================
# 5. LIMPEZA (opcional)
# ============================================================================
print("\n🧹 Limpando arquivos temporários...")

# Perguntar se deseja manter os frames individuais
resposta = input("Deseja manter os frames individuais? (s/n): ").lower()

if resposta != 's':
    # Remover frames temporários
    for i in range(len(frames_data)):
        frame_path = os.path.join(frames_dir, f"frame_{i:03d}.png")
        if os.path.exists(frame_path):
            os.remove(frame_path)
    
    # Remover diretório
    try:
        os.rmdir(frames_dir)
        print("✅ Limpeza concluída!")
    except:
        print("⚠️ Não foi possível remover o diretório de frames")
else:
    print(f"✅ Frames individuais mantidos em: {frames_dir}")

# ============================================================================
# 6. INFORMAÇÕES FINAIS
# ============================================================================
print("\n📊 Resumo da animação criada:")
print(f"- Total de pentadas processadas: {len(frames_data)}")
print(f"- Duração por frame: 0.5 segundos")
print(f"- Duração total do GIF: ~{len(frames_data) * 0.5:.1f} segundos")
print(f"- Arquivo de saída: {output_path}")

print("\n💡 Dicas para personalização:")
print("1. Para acelerar/desacelerar: ajuste 'duration' (ms por frame)")
print("2. Para maior qualidade: aumente 'dpi' ao salvar frames")
print("3. Para arquivo menor: reduza 'dpi' ou use compressão")
print("4. Para pausar em pentadas específicas: duplique frames importantes")
print("\n🎉 Processo concluído com sucesso!")

2025-10-17 15:09:13,270 - loczcit_iqr.core.data_loader_era5 - INFO - Cliente CDS inicializado com credenciais fornecidas
2025-10-17 15:09:13,271 - loczcit_iqr.core.data_loader_era5 - INFO - ERA5DataLoader inicializado (cache: era5_cache)
2025-10-17 15:09:13,274 - loczcit_iqr.core.processor - INFO - Arquivo padrão da área de estudo encontrado: E:\ambiente_python_2025-main\src\data\shapefiles\Area_LOCZCIT.parquet
2025-10-17 15:09:13,275 - loczcit_iqr.core.iqr_detector - INFO - IQRDetector inicializado com constante: 0.75
2025-10-17 15:09:13,276 - loczcit_iqr.core.spline_interpolator - INFO - SplineInterpolator inicializado: método='bspline', lat_ref=0.00°
Carregando dados ERA5 para o ano 2024...
2025-10-17 15:09:13,277 - loczcit_iqr.core.data_loader_era5 - INFO - Carregando dados ERA5: 2024-01-01 a 2024-12-31
2025-10-17 15:09:13,278 - loczcit_iqr.core.data_loader_era5 - INFO - Usando arquivo do cache: era5_cache\era5_olr_noaa_format_20240101_20241231_area_17.00_-80.00_-12.00_4.00.nc
2025

Processando pentadas:   0%|          | 0/73 [00:00<?, ?it/s]

2025-10-17 15:09:13,482 - loczcit_iqr.core.processor - INFO - Buscando mínimos de OLR (método: column_minimum, threshold: None, raio: 1px)
2025-10-17 15:09:13,483 - loczcit_iqr.core.processor - INFO - Carregando área de estudo de: E:\ambiente_python_2025-main\src\data\shapefiles\Area_LOCZCIT.parquet
2025-10-17 15:09:13,500 - loczcit_iqr.core.processor - INFO - Encontradas 16 coordenadas de mínimo
2025-10-17 15:09:13,501 - loczcit_iqr.core.iqr_detector - INFO - Detectando outliers em 16 coordenadas (analisando latitude)
2025-10-17 15:09:13,501 - loczcit_iqr.core.iqr_detector - INFO - Detecção concluída: 15 válidas, 1 outliers
2025-10-17 15:09:13,502 - loczcit_iqr.core.processor - INFO - Buscando mínimos de OLR (método: local_minimum, threshold: 230, raio: 1px)
2025-10-17 15:09:13,503 - loczcit_iqr.core.processor - INFO - Aplicando threshold OLR <= 230 W/m²
2025-10-17 15:09:13,503 - loczcit_iqr.core.processor - INFO - Carregando área de estudo de: E:\ambiente_python_2025-main\src\data\sh

Processando pentadas:   4%|▍         | 3/73 [00:00<00:02, 25.11it/s]

2025-10-17 15:09:13,602 - loczcit_iqr.core.processor - INFO - Buscando mínimos de OLR (método: column_minimum, threshold: None, raio: 1px)
2025-10-17 15:09:13,602 - loczcit_iqr.core.processor - INFO - Carregando área de estudo de: E:\ambiente_python_2025-main\src\data\shapefiles\Area_LOCZCIT.parquet
2025-10-17 15:09:13,620 - loczcit_iqr.core.processor - INFO - Encontradas 8 coordenadas de mínimo
2025-10-17 15:09:13,620 - loczcit_iqr.core.iqr_detector - INFO - Detectando outliers em 8 coordenadas (analisando latitude)
2025-10-17 15:09:13,621 - loczcit_iqr.core.iqr_detector - INFO - Detecção concluída: 8 válidas, 0 outliers
2025-10-17 15:09:13,622 - loczcit_iqr.core.processor - INFO - Buscando mínimos de OLR (método: local_minimum, threshold: 230, raio: 1px)
2025-10-17 15:09:13,623 - loczcit_iqr.core.processor - INFO - Aplicando threshold OLR <= 230 W/m²
2025-10-17 15:09:13,623 - loczcit_iqr.core.processor - INFO - Carregando área de estudo de: E:\ambiente_python_2025-main\src\data\shape

...


Iniciando a criação da visualização completa...
✅ Visualização completa pronta.


Salvando frames:  99%|█████████▊| 72/73 [00:43<00:00,  1.70it/s]


Iniciando a criação da visualização completa...
✅ Visualização completa pronta.


Salvando frames: 100%|██████████| 73/73 [00:44<00:00,  1.66it/s]



🎬 Criando GIF animado...

✅ GIF salvo como: zcit_evolution_2024_all_pentads.gif

🧹 Limpando arquivos temporários...
✅ Limpeza concluída!

📊 Resumo da animação criada:
- Total de pentadas processadas: 73
- Duração por frame: 0.5 segundos
- Duração total do GIF: ~36.5 segundos
- Arquivo de saída: zcit_evolution_2024_all_pentads.gif

💡 Dicas para personalização:
1. Para acelerar/desacelerar: ajuste 'duration' (ms por frame)
2. Para maior qualidade: aumente 'dpi' ao salvar frames
3. Para arquivo menor: reduza 'dpi' ou use compressão
4. Para pausar em pentadas específicas: duplique frames importantes

🎉 Processo concluído com sucesso!
